# 서울시 구별 CCTV 현황 분석하기

### 본 장은 ["민형기 저자의 파이썬으로 데이터 주무르기"](https://pinkwink.kr) 1장 내용입니다.

## 분석 목표
* 서울시 각 구별 CCTV수를 파악하고, 인구대비 CCTV 비율을 파악해서 순위 비교
* 인구대비 CCTV의 평균치를 확인하고 그로부터 CCTV가 과하게 부족한 구를 확인.

## 파이썬 공부 목표
* JupyterLab 사용환경을 익히고 Python 기본 문법을 확인
* Pandas와 Matplotlib의 기본적 사용법을 확인
* 단순한 그래프 표현에서 한 단계 더 나아가 경향을 확인하고 시각화하는 기초 확인

In [1]:
# pandas 패키지 불러오기
import pandas as pd

In [5]:
# 자료 불러오기
CCTV_Seoul = pd.read_csv("CCTV_Seoul.csv", encoding = "utf-8")
CCTV_Seoul.head()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


In [15]:
# 컬럼명 조회
CCTV_Seoul.columns

Index(['기관명', '소계', '2013년도 이전', '2014년', '2015년', '2016년'], dtype='object')

In [37]:
# 첫 번째 컬럼명 조회
CCTV_Seoul.comlumns[0]

AttributeError: 'DataFrame' object has no attribute 'comlumns'

In [36]:
# 첫 번째 컬럼명을 '기관명'에서 '구별'로 바꾸는 것이 좋을 듯함.
CCTV_Seoul.rename(columns = {CCTV_Seoul.columns[0] : '구별'}, inplace=True)
CCTV_Seoul.head

<bound method NDFrame.head of       구별    소계  2013년도 이전  2014년  2015년  2016년
0    강남구  3238       1292    430    584    932
1    강동구  1010        379     99    155    377
2    강북구   831        369    120    138    204
3    강서구   911        388    258    184     81
4    관악구  2109        846    260    390    613
5    광진구   878        573     78     53    174
6    구로구  1884       1142    173    246    323
7    금천구  1348        674     51    269    354
8    노원구  1566        542     57    451    516
9    도봉구   825        238    159     42    386
10  동대문구  1870       1070     23    198    579
11   동작구  1302        544    341    103    314
12   마포구   980        314    118    169    379
13  서대문구  1254        844     50     68    292
14   서초구  2297       1406    157    336    398
15   성동구  1327        730     91    241    265
16   성북구  1651       1009     78    360    204
17   송파구  1081        529     21     68    463
18   양천구  2482       1843    142     30    467
19  영등포구  1277        495    2

## 2-2. 엑셀파일 읽기 - 서울시 인구현황

In [24]:
# 엑셀로 미리 행정동을 제거하고, 노인인구를 넣는다.
# (데이터에 맞게 바뀐 코드)
pop_Seoul = pd.read_csv("newPop_Seoul.csv")
pop_Seoul.head

<bound method NDFrame.head of           Col     col1      col2     col3     col4     col5     col6     col7  \
0       Seoul  4220082  10124579  4957857  5166722  9857426  4830206  5027220   
1      Jongro    73594    164257    80094    84163   154770    75967    78803   
2       Chung    60412    134593    66337    68256   125709    62253    63456   
3        Yong   107666    244444   119423   125021   229161   110878   118283   
4   Seoundong   132902    312711   154077   158634   304808   150368   154440   
5       Kwang   160798    372298   180645   191653   357703   174414   183289   
6         DDM   159938    366011   181185   184826   350647   175324   175323   
7    Chunrang   179132    412780   205125   207655   408226   203325   204901   
8   Seoungbuk   187112    455407   221103   234304   444055   216556   227499   
9     Gangbuk   142533    328002   160252   167750   324479   158918   165561   
10     Dobong   137378    346234   169553   176681   344166   168759   175407  

In [38]:
pop_Seoul.rename(columns = {pop_Seoul.columns[0]: '구별', 
                            pop_Seoul.columns[2]: '총인구',
                            pop_Seoul.columns[5]: '내국인',
                            pop_Seoul.columns[8]: '외국인',
                            pop_Seoul.columns[13]: '고령자',
                           }, inplace=True)
pop_Seoul.head()

,구별,col1,총인구,col3,col4,내국인,col6,col7,외국인,col9,col10,col11,col12,고령자,col14,col15
0,Seoul,4220082,10124579,4957857,5166722,9857426,4830206,5027220,267153,127651,139502,2.335838,NaN,1365126,602107,763019
1,Jongro,73594,164257,80094,84163,154770,75967,78803,9487,4127,5360,2.103025,NaN,26182,11441,14741
2,Chung,60412,134593,66337,68256,125709,62253,63456,8884,4084,4800,2.080861,NaN,21384,9332,12052
3,Yong,107666,244444,119423,125021,229161,110878,118283,15283,8545,6738,2.128444,NaN,36882,15489,21393
4,Seoundong,132902,312711,154077,158634,304808,150368,154440,7903,3709,4194,2.293479,NaN,41273,17972,23301


In [42]:
# 엑셀을 열고 같이 확인해보면서 usecols를 선택해야 한다. 구분(B), 총인구(D), 내국인(G), 외국인(J), 65세 이상 총인구(O) 선택P
pop_Seoul = pop_Seoul.loc[:,["구별", "총인구", "내국인", "외국인", "고령자"]]
pop_Seoul.head()

,구별,총인구,내국인,외국인,고령자
0,Seoul,10124579,9857426,267153,1365126
1,Jongro,164257,154770,9487,26182
2,Chung,134593,125709,8884,21384
3,Yong,244444,229161,15283,36882
4,Seoundong,312711,304808,7903,41273


In [ ]:
# 0번째 열 삭제 
pop_Seoul = pop_Seoul.iloc[:,1:]
pop_Seoul.head()

# 3. Pandas 기초

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 컬럼 슬라이싱


### loc 옵션:(행번호 + 열이름으로 조회해보고 싶을 때)
- loc는 location 옵션으로 슬라이싱할 때 loc 옵션을 이용해서 위치 값을 지정할 수 있다.
- [행,열] 구조로 이루어지며, 행은 숫자로, 열은 문자로 지정해야 한다.
- 복수의 열을 지정시 ['A', 'B', 'C']의 구조로 만들어서 pd.loc[:, ['A','B','C']] 구조가 되어야 한다.

In [ ]:
# 인덱스는 모두(:), 컬럼은 ['구별','인구수']만


In [ ]:
# 두번째행부터 10번째까지


In [ ]:
# 하나의 행만


### iloc 옵션(행번호 + 열번호로 조회하고 싶을 때)
- iloc는 integer location 옵션으로 loc와 달리 행과 열의 번호를 이용해서 데이터에 바로 접근할 때 사용한다.
- iloc을 사용하면 행이나 열의 범위를 지정하면 된다. 특히 콜론(:)을 사용하면 전체를 의미한다.?

In [ ]:
# 복수의 행, 열을 선택할 때는 내부에 [] 구조로 만들어야 한다.


### 쿼리를 통한 데이터 검색
- 키워드에 데이터를 조회해오는 .query()

In [1]:
# 개별 검색


### 조건을 만족하는 데이터 검색

In [2]:
# 전체 데이터에서 조건에 해당되지 않으면 NaN 처리함.


In [3]:
# 인구수가 40만명을 넘는가? 넘는 것만 남기기


### 새로운 열을 생성하기
- 새로운 열을 만들 때는 data['새 열이름'] = 값

In [4]:
# pop_Seoul2라는 이름으로 pop_Seoul 복사하기


In [5]:
# 새로운 열을 만들 때는 data['새 열이름'] = 값


- isin()함수는 데이터에 해당값이 있는지를 참거짓으로 반환한다.

#### 참이 나오는 전체 행을 조회하기

In [6]:
# isin()함수


### One-line 함수 lambda
- lambda는 함수를 생성할 때 사용하는 예약어로, def와 동일한 역할을 한다. 
- 보통 함수를 한줄로 간결하게 만들 때 사용한다. 우리말로는 "람다"라고 읽고 def를 사용해야 할 정도로 복잡하지 않거나 def를 사용할 수 없는 곳에 주로 쓰인다.
- lambda 인수1, 인수2, ... : 인수를 이용한 표현식

In [7]:
# 각 열 마다의 최대값과 최소값의 차이를 구해줌


# 4. CCTV 데이터 파악하기

In [9]:
# 구별 CCTV 데이터에서 CCTV 전체 개수인 소계로 정렬하기1


##### CCTV의 전체 개수가 가장 작은 구는 '도봉구', '강북구', '광진구', '강서구', '중랑구' 순임.
##### 특이점은 강남 3구중 하나인 '송파구'가 뒤에서 9위에 랭크

In [10]:
# 구별 CCTV 데이터에서 CCTV 전체 개수인 소계로 정렬하기2

# ascending이 False이면 오름차순(큰 것부터 시작)

##### 
##### 

#### 조금 더 쪼개어 살펴보자. 
##### 최근증가율 = 2014년부터 2016년까지 최근 3년간 CCTV 수 / 2013년 이전 CCTV 수 * 100

##### 

# 5. 서울시 인구 데이터 파악하기

In [ ]:
pop_Seoul.head()

In [ ]:
# 서울시 전체 데이터인 첫행을 날린다.
# 0번째 행인 서울시 전체 데이터를 날림.


In [ ]:
# unique 함수는 1번 이상 나타난 데이터를 식별한다.


In [ ]:
# isnull 함수를 사용하여 NaN 데이터를 추출해낼 수 있다.


In [ ]:
# drop 함수를 사용하여 NaN이 있는 행을 삭제할 수 있다.
# pop_Seoul.drop([26], inplace=True)
# pop_Seoul.head()

##### 데이터 추가
- 분석에 필요한 데이터를 추가한다.
- 외국인 비율 & 고령자 비율

In [ ]:
# 인구수로 정렬


In [ ]:
# 외국인수로 정렬


In [ ]:
# 외국인비율로 정렬


In [ ]:
# 고령자로 정렬


In [ ]:
# 고령자비율로 정렬


# 6. Pandas 고급 두 DataFrame 병합하기

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], 
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])

In [ ]:
# 연습용 데이터1
df1

In [ ]:
# 연습용 데이터2
df2

In [ ]:
# 연습용 데이터3
df3

### Concat 함수
- 특정 축을 따라 Pandas 객체를 연결하고, 다른 축을 따라 선택적 집합 논리를 연결한다.
- 연결 축에 계층적 색인 계층을 추가 할 수도 있다. 레이블이 전달된 축 번호와 동일하거나 겹치는 경우 유용 할 수 있다.

In [ ]:
# key 옵션을 통해 x,y,z라는 대분류를 지정할 수 있음.


In [ ]:
# 레벨0


In [ ]:
# 레벨1


In [ ]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'], 
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])

In [ ]:
df1

In [ ]:
df4

In [ ]:
# 종으로 연결


In [ ]:
# 횡으로 연결


In [ ]:
# 횡으로 연결하는데 NaN이 없는 모든 데이터가 있는 영역만 합치기


In [ ]:
# df1의 인덱스에 맞추어 df4를 합침.


In [ ]:
# 두 데이터를 열 방향으로 index를 무시하고 합친 후 다시 index를 부여한다.


In [ ]:
left = pd.DataFrame({'key': ['K0', 'K4', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [ ]:
left

In [ ]:
right

In [ ]:
# key를 기준으로 합친다.


### merge 함수
![그룹연산](data join50.png)

In [ ]:
pd.merge(left, right, how='inner', on='key')

In [ ]:
pd.merge(left, right, how='outer', on='key')

In [ ]:
pd.merge(left, right, how='left', on='key')

In [ ]:
pd.merge(left, right, how='right', on='key')

# 7. CCTV 데이터와 인구 데이터 합치고 분석하기

In [ ]:
# '구별'을 중심으로 합치기


In [ ]:
# 연도별 인구수는 모두 제거


In [ ]:
# 그래프 그리기를 위한 set_index


### 상관분석(Correlation Analysis)
- 2개열의 데이터간의 상관관계를 분석하는 것으로, 2개열이 모두 숫자형 데이터로 이루어져있어야 한다.
![상관분석1](Corr1.jpg)
![상관분석2](Corr2.jpg)

##### 

##### 

##### 
##### 

# 8. 그래프 그리기 기초 - matplotlib

In [ ]:
# 그래프를 그리는 라이브러리
import matplotlib.pyplot as plt
# 그래프의 결과를 출력 세션에 나타나게 하는 설정임.
%matplotlib inline 

In [ ]:
plt.figure()
plt.plot([1,2,3,4,5,6,7,8,9,8,7,6,5,4,3,2,1,0], lw=3, label = 'pyramid')
plt.grid() # 그리드 적용하기
plt.xlabel('X축')       # x축 라벨 적용하기
plt.ylabel('Y축')  # y축 라벨 적용하기
plt.title('Example of plotting') # 타이틀
plt.legend()
plt.show()

# 9. CCTV와 인구현황 그래프로 분석하기

#### 한글 폰트문제 해결
- matplotlib이 기본으로 가진 폰트는 한글을 지원하지 않기 때문에 matplotlib의 폰트를 변경할 필요가 있다.
- Mac인지 Windows인지 나누어 지원

In [ ]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 

In [ ]:
data_result.head()

In [ ]:
plt.figure()
data_result['소계'].plot(kind='barh', grid=True, figsize=(10,10)) # 수평 막대 그래프를, 그리드를 사용하여, 그림 크기를 10*10으로 지정함.
plt.show()

In [ ]:
# sort_values 사용
data_result['소계'].sort_values().plot(kind='barh', 
                                     grid=True, figsize=(10,10))
plt.show()

##### 

In [ ]:
# 인구 대비 CCTV 비율을 구한다.
data_result['CCTV비율'] = data_result['소계'] / data_result['인구수'] * 100

# CCTV 비율을 가지고 수평 막대 그래프를 그린다.
data_result['CCTV비율'].sort_values().plot(kind='barh', 
                                         grid=True, figsize=(10,10))
plt.show()

##### 
##### 
##### 

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(data_result['인구수'], data_result['소계'], s=50)
plt.xlabel('인구수')
plt.ylabel('CCTV')
plt.title("서울 자치구별 인구수와 CCTV와의 상관관계 산점도")
plt.grid()
plt.show()

#### 위 데이터를 대표하는 직선을 하나 그려보자.
- polyfit과 poly1d 함수를 사용하여 (x, y_linear)와 (x, y_quadratic) 좌표 쌍들에 대해 선형 및 2차 다항식을 만든다.
- polyfit 함수를 이용해 지정한 특정 차수의 다항식 계수를 계산하고,
- poly1d함수로 그 계수를 사용하여 실제 다항식을 만든다.

In [ ]:
# polyfit 함수를 이용해 지정한 특정 차수의 다항식 계수를 계산하고,
fp1 = np.polyfit(data_result['인구수'], data_result['소계'], 1)
fp1

In [ ]:
# poly1d함수로 그 계수를 사용하여 실제 다항식을 만든다.
f1 = np.poly1d(fp1) # y축
fx = np.linspace(100000, 700000, 100) # x축

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(data_result['인구수'], data_result['소계'], s=50)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='r')
plt.xlabel('인구수')
plt.ylabel('CCTV')
plt.grid()
plt.show()

# 10. 조금더 설득력 있는 자료 만들기

In [ ]:
fp1 = np.polyfit(data_result['인구수'], data_result['소계'], 1)

f1 = np.poly1d(fp1) # y축
fx = np.linspace(100000, 700000, 100) # x축

# 오차를 추가한다.
data_result['오차'] = np.abs(data_result['소계'] - f1(data_result['인구수']))

df_sort = data_result.sort_values(by='오차', ascending=False)
df_sort.head()

In [ ]:
plt.figure(figsize=(14,10))
plt.scatter(data_result['인구수'], data_result['소계'], 
            c=data_result['오차'], s=50)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='r')

for n in range(10):
    plt.text(df_sort['인구수'][n]*1.02, df_sort['소계'][n]*0.98, 
             df_sort.index[n], fontsize=15)
    
plt.xlabel('인구수')
plt.ylabel('인구당비율')
plt.colorbar()
plt.grid()
plt.show()

* 
* 
* 